In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [39]:
# Get the table from wiki html page
postcode_df = pd.read_html(url)[0]
postcode_df.columns = postcode_df.iloc[0,:]
postcode_df = postcode_df.iloc[1:, :]
postcode_df.head(10)

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


Clean the data frame.

In [41]:
# Filtering out the rows which have a Not assigned in Borough
postcode_df = postcode_df[postcode_df["Borough"]!="Not assigned"]

# Assign Neighbourhood of the rows which have Not assigned in Neighbourhood but not in Borough as the content of Borough
idx = postcode_df[postcode_df["Neighbourhood"]=="Not assigned"].index[0]
postcode_df.at[idx, "Neighbourhood"] = postcode_df.loc[[idx]]["Borough"].iloc[0]

postcode_df.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


Combine the rows which have duplicated postcodes.

In [43]:
# Combine the duplicated postcodes
postcode_df_list = []

for postcode in postcode_df["Postcode"].unique():
    postcode_df_sub = postcode_df[postcode_df["Postcode"] == postcode]
    neighbors = postcode_df_sub["Neighbourhood"].tolist()
    neighborStr = ", ".join(neighbors)
    postcode_df_list.append([postcode, postcode_df_sub["Borough"].iloc[0], neighborStr])

postcode_df_comb = pd.DataFrame(postcode_df_list, columns = postcode_df.columns)
postcode_df_comb.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [44]:
postcode_df_comb.shape

(103, 3)